## Generate and use test trade data

Here we use the RTS 2 Annex 3 model to generate test trade data which we then classify.

In these first examples we generate a couple of really simple cases.  Read the comments in the code which explain what  is happening at each step.

Do run the steps in order.  The later steps build on work done by the earlier steps ... if you skip a step something may break.  If that happens, just start again from the top.

In [1]:
# Import the RTS 2 Annex III taxonomy 
# ... and get hold of the object which represents the root of the taxonomy tree.
import rts2_annex3
root = rts2_annex3.class_root

# Now we print the root object to see what it is.
print(root)

In [11]:
# Using the taxonomy to generate some test data:

# Starting with a Sub-asset class which has no criteria:
sub_asset_class = root.sub_asset_class_by_name("Exchange Traded Notes (ETNs)")

# Ask the Sub-asset class to generate a single sample trade
sample_trades = sub_asset_class.make_test_samples(number=1)
sample_trade = sample_trades[0]

# Get the RTS 2 Annex III taxonomy to classify the trade
classification = root.classification_for(subject=sample_trade)

print(classification.classification_dict())

OrderedDict([('RTS2 version', 'EU 2017/583 of 14 July 2016'), ('Asset class', 'Bonds (ETC and ETN bond types)'), ('Sub-asset class', 'Exchange Traded Notes (ETNs)')])


In [12]:
# Note that model is implemented in Python and the printed result above is 
# simply the literal form of a Python OrderedDict.

# We can make the classification easier to understand using JSON.
# ... but note that the content is just the same as above.

import json
print(json.dumps(classification.classification_dict(), indent=4))


{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Notes (ETNs)"
}


In [16]:
# Now we generate a sample trade for an Asset class.

# We select an Asset class which has two child Sub-asset classes, including
# the Sub-asset class used as an example above.
asset_class = root.asset_class_by_name("Bonds (ETC and ETN bond types)")

# When we ask the asset class to make a sample it randomly chooses one of
# its two children.
sample = asset_class.make_test_samples(1)[0]

# This new sample trade is classified.
classification = root.classification_for(subject=sample)
print(json.dumps(classification.classification_dict(), indent=4))

# Run the this cell many times and note that the Sub-asset class is selected randomly


{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Commodities (ETCs)"
}


In [36]:
# The examples above make sample trades for Sub-asset classes which have no
# criteria.  Now we make a sample trade which has many criteria.  First 
# get hold of the Sub-asset class
sub_asset_class = root.sub_asset_class_by_name("Energy commodity swaps")

# Then generate a sample trade
sample_trade = sub_asset_class.make_test_samples(1)[0]

# Notice that the sample trade has some values which change randomly, e.g. to_date
print(vars(sample_trade))

# ... so when the trade is classified the maturity bucket criteion varies
classification = root.classification_for(subject=sample_trade)
print(json.dumps(classification.classification_dict(), indent=4))


{'energy_type': 'LGHT', 'to_date': datetime.date(2018, 5, 21), 'from_date': datetime.date(2018, 4, 21), 'underlying_energy': 'underlying_energy.value', 'settlement_type': 'settlement_type.value', 'load_type': 'load_type.value', 'asset_class_name': 'Commodity Derivatives', 'sub_asset_class_name': 'Energy commodity swaps', 'notional_currency': 'notional_currency.value', 'delivery': 'delivery.value'}
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Commodity Derivatives",
    "Sub-asset class": "Energy commodity swaps",
    "Segmentation criterion 1 description": "energy type: oil, oil distillates, coal, oil light ends, natural gas, electricity, inter-energy",
    "Segmentation criterion 1": "LGHT",
    "Segmentation criterion 2 description": "underlying energy",
    "Segmentation criterion 2": "underlying_energy.value",
    "Segmentation criterion 3 description": "notional currency defined as the currency in which the notional amount of the swap is denominated",
 

In [41]:
# Now to generate and clasify lots of sample trades

# Generate a sample trade for each Sub-asset class in the taxonomy
samples = []
for sub_asset_class in root.all_sub_asset_classes():
    samples.extend(sub_asset_class.make_test_samples(1))

# Classify each sample
classifications = [root.classification_for(subject=sample) for sample in samples]

# Show a couple of random examples of the classifications
import random
for n in range(2):
    a_classification = random.choice(classifications)
    print("\nSample trade {n} looks like this:".format(n=n))
    print(vars(a_classification.subject))
    print("\nThe classification of sample trade {n} looks like this:".format(n=n))
    print(json.dumps(a_classification.classification_dict(), indent=4))



Sample trade 0 looks like this:
{'asset_class_name': 'Financial contracts for differences (CFDs)', 'sub_asset_class_name': 'Other CFDs'}

The classification of sample trade 0 looks like this:
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Financial contracts for differences (CFDs)",
    "Sub-asset class": "Other CFDs"
}

Sample trade 1 looks like this:
{'energy_type': 'INRG', 'to_date': datetime.date(2019, 4, 21), 'from_date': datetime.date(2018, 4, 21), 'underlying_energy': 'underlying_energy.value', 'load_type': 'load_type.value', 'asset_class_name': 'Commodity Derivatives', 'sub_asset_class_name': 'Energy commodity futures/forwards', 'notional_currency': 'notional_currency.value', 'delivery': 'delivery.value'}

The classification of sample trade 1 looks like this:
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Commodity Derivatives",
    "Sub-asset class": "Energy commodity futures/forwards",
    "Segmentation criterion 1 descript